In [1]:
# (comment) import necessary modules
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch import cuda

from torchvision import datasets
from torchvision import transforms

In [2]:
# load or download MNIST datasets
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.7,), (0.7,)),]) # transform image data to tensors and normalize them (mean=0.7, std=0.7)
train_data = datasets.MNIST(root='./data', train=True, download=True, transform=trans)
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=trans)

In [3]:
# create data loaders
batch_size = 64
train_dataloader = DataLoader(train_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [4]:
# check the data from the data loader
for X, y in train_dataloader:
    print(f'Shape of X [batch, channel, height, width]: {X.shape} ({X.dtype})') 
    print(f'Shape of y: {y.shape} ({y.dtype})')
    break

Shape of X [batch, channel, height, width]: torch.Size([64, 1, 28, 28]) (torch.float32)
Shape of y: torch.Size([64]) (torch.int64)


In [5]:
# define MLP network with one hidden layer (original version)
# run one of this or below cell
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.flatten = nn.Flatten() # flatten the input tensor as a 1D vector ((28, 28) -> (784))
        self.input_layer = nn.Linear(28*28, 512)
        self.hidden_layer = nn.Linear(512, 256)
        self.output_layer = nn.Linear(256, 10)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.flatten(x)
        h = self.relu(self.input_layer(x))
        h = self.relu(self.hidden_layer(h))
        y = self.output_layer(h)
        return y

In [6]:
# define MLP network (simple version)
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_layers = nn.Sequential(nn.Linear(28*28, 256),
                                                nn.ReLU(),
                                                nn.Linear(256, 512),
                                                nn.ReLU(),
                                                nn.Linear(512, 10))
    
    def forward(self, x):
        return self.linear_relu_layers(self.flatten(x))

In [7]:
# check device for training
device = 'cuda' if cuda.is_available() else 'cpu'
print(f'device: {device}')

device: cuda


In [8]:
# get the MLP model and send it to the device
myMLP = MLP().to(device)
print(myMLP)

MLP(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_layers): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [9]:
# define a loss function and an optimizer
loss_fn = nn.CrossEntropyLoss() # loss function for classification
optimizer = torch.optim.SGD(myMLP.parameters(), lr=5e-3) # stochastic gradient descent with learning rate of 0.005
print(loss_fn)
print(optimizer)

CrossEntropyLoss()
SGD (
Parameter Group 0
    dampening: 0
    lr: 0.005
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [10]:
# train the MLP model
epochs = 5
ndata = len(train_dataloader.dataset) 
print('training starts!')

for e in range(epochs):
    print(f'\nepoch {e+1}\n------------------------------')
    myMLP.train() # train mode
    
    for b, (X, y) in enumerate(train_dataloader):
        X, y = X.to(device), y.to(device) # input and target to device(gpu)

        prediction = myMLP(X) # forward pass
        train_loss = loss_fn(prediction, y) # calculate the loss 

        optimizer.zero_grad() # clear gradients
        train_loss.backward() # backpropagation
        optimizer.step() # update the parameters

        if b % 100 == 0: # track the training
            train_loss, current = train_loss.item(), b * len(X) 
            print(f'loss: {train_loss:>7f}  [{current:>5d}/{ndata:>5d}]')
print('\ntraining is finished!')

training starts!

epoch 1
------------------------------
loss: 2.291759  [    0/60000]
loss: 2.223994  [ 6400/60000]
loss: 2.168183  [12800/60000]
loss: 2.010625  [19200/60000]
loss: 1.827311  [25600/60000]
loss: 1.603575  [32000/60000]
loss: 1.267062  [38400/60000]
loss: 1.254340  [44800/60000]
loss: 1.026646  [51200/60000]
loss: 0.822613  [57600/60000]

epoch 2
------------------------------
loss: 0.843861  [    0/60000]
loss: 0.688846  [ 6400/60000]
loss: 0.673070  [12800/60000]
loss: 0.608711  [19200/60000]
loss: 0.542259  [25600/60000]
loss: 0.528805  [32000/60000]
loss: 0.451171  [38400/60000]
loss: 0.574159  [44800/60000]
loss: 0.559926  [51200/60000]
loss: 0.499349  [57600/60000]

epoch 3
------------------------------
loss: 0.459933  [    0/60000]
loss: 0.368058  [ 6400/60000]
loss: 0.404581  [12800/60000]
loss: 0.448154  [19200/60000]
loss: 0.355247  [25600/60000]
loss: 0.411089  [32000/60000]
loss: 0.324254  [38400/60000]
loss: 0.464898  [44800/60000]
loss: 0.462106  [51200/

In [11]:
# test the MLP model
ndata = len(test_dataloader.dataset)
nbatch = len(test_dataloader)
myMLP.eval() # test mode
test_loss, correct = 0, 0 # initialize

with torch.no_grad():
    for X, y in test_dataloader:
        X, y = X.to(device), y.to(device)
        prediction = myMLP(X)
        test_loss += loss_fn(prediction, y).item() # add up the loss
        correct += (prediction.argmax(1) == y).type(torch.float).sum().item() # add up the correct predictions
test_loss /= nbatch # average the loss
correct /= ndata # accuracy for all data
print(f"test error\n-> accuracy: {(100*correct):>0.1f}%, average loss: {test_loss:>8f}")

test error
-> accuracy: 90.6%, average loss: 0.320947
